In [1]:
import glob

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.markers import CARETLEFTBASE, CARETRIGHTBASE
from csv_dtypes import column_dtypes

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn import svm, linear_model

In [2]:
all_files = glob.glob("../roll_data/*-item_creation.csv")

df = pd.concat((pd.read_csv(f, dtype=column_dtypes) for f in all_files), ignore_index=True)

# seasons = [11, 12]
seasons = [15,16,17,18]
# seasons = [14]
# seasons = [13, 14]
df = df[df['season'].isin(seasons)]
len(df)

416

In [3]:
df


,season,day,roll_type,category,element,roll,prefix_index
0,15,13,GLITTER,post_prefix,Leg,0.564122,-1
1,15,13,GLITTER,prefix,Plant-Based,0.606256,0
2,15,15,GLITTER,post_prefix,Leg,0.507357,-1
3,15,30,GLITTER,prefix,Inflatable,0.421802,0
4,15,41,GLITTER,suffix,Vanity,0.525603,-1
...,...,...,...,...,...,...,...
411,18,87,GLITTER,prefix,Flickering,0.120699,0
412,18,94,PRIZE,prefix,Repeating,0.278428,0
413,18,94,PRIZE,prefix,Golden,0.083260,1
414,18,114,PRIZE,post_prefix,Rock,0.138951,-1


In [4]:
dfpostp = df[(df['category'] == 'post_prefix')]

In [5]:
a = dfpostp[dfpostp['roll_type'] != 'GLITTER'].groupby(['element'])['roll'].agg(["min", "max", "count"]).sort_values(by="min")
a['upper_bound'] = [n/8 for n in range(1,9)] 
a

,min,max,count,upper_bound
element,,,,
Skate,0.033127,0.118001,6,0.125
Rock,0.131308,0.224418,14,0.250
Concrete,0.252699,0.374949,8,0.375
Arm,0.389671,0.490214,11,0.500
Leg,0.502263,0.612864,13,0.625
Head,0.655927,0.748809,6,0.750
Blood,0.774566,0.852702,12,0.875
Air,0.881882,0.991579,12,1.000


In [8]:
dfpostp[dfpostp['roll_type'] == 'GLITTER'].sort_values(['season', 'day'])

,season,day,roll_type,category,element,roll,prefix_index
0,15,13,GLITTER,post_prefix,Leg,0.564122,-1
2,15,15,GLITTER,post_prefix,Leg,0.507357,-1
7,15,61,GLITTER,post_prefix,Leg,0.569808,-1
15,16,18,GLITTER,post_prefix,Plastic,0.684670,-1
56,16,61,GLITTER,post_prefix,Paper,0.507500,-1
137,17,4,GLITTER,post_prefix,Spirit,0.768028,-1
140,17,6,GLITTER,post_prefix,Spirit,0.714709,-1
358,18,57,GLITTER,post_prefix,Blood,0.880519,-1
409,18,85,GLITTER,post_prefix,Leg,0.207278,-1


In [9]:
dfsuf = df[(df['category'] == 'suffix') & (df['season'] != 15)]
dfsuf

,season,day,roll_type,category,element,roll,prefix_index
19,16,29,CHEST,suffix,Wisdom,0.697582,-1
29,16,29,CHEST,suffix,Blaserunning,0.074754,-1
34,16,29,CHEST,suffix,Strength,0.548553,-1
41,16,29,CHEST,suffix,Wisdom,0.749393,-1
50,16,29,CHEST,suffix,Strength,0.575892,-1
...,...,...,...,...,...,...,...
378,18,80,CHEST,suffix,Blaserunning,0.014805,-1
385,18,80,CHEST,suffix,Intelligence,0.479673,-1
391,18,80,CHEST,suffix,the Famine,0.937017,-1
397,18,80,CHEST,suffix,Strength,0.561177,-1


In [10]:
a = dfsuf.groupby(['element'])['roll'].agg(["min", "max", "count"]).sort_values(by="min")
a['upper_bound'] = [n/12 for n in range(1,13)] 
a

,min,max,count,upper_bound
element,,,,
Blaserunning,0.014805,0.081568,7,0.083333
Fourtitude,0.094812,0.128737,4,0.166667
Stamina,0.169266,0.246766,8,0.250000
Vanity,0.254345,0.254345,1,0.333333
Vitality,0.348437,0.393270,6,0.416667
Intelligence,0.431827,0.492013,9,0.500000
Strength,0.502168,0.575892,7,0.583333
Charisma,0.597204,0.660468,7,0.666667
Wisdom,0.668891,0.749393,8,0.750000


In [11]:
dfpre = df[(df['category'] == 'prefix') & (df['prefix_index'] == 0)]

In [12]:
dfpre['sd'] = [(row[1]['season'], row[1]['day']) for row in dfpre.iterrows()]

a = dfpre[dfpre['roll_type'] != 'GLITTER'].groupby(['element'])['roll', 'sd'].agg(["min", "max", "count"]).sort_values(by=[("roll", "min")])
a['diff'] = a[('roll', 'min')] - a[('roll', 'max')].shift()
a['lower_bound'] = [n/len(a) for n in range(0,len(a))] 
a['upper_bound'] = [n/len(a) for n in range(1,len(a)+1)] 
a['lb'] = a['lower_bound'] <= a[('roll', 'min')]
a['ub'] = a['upper_bound'] >= a[('roll', 'max')]
print(len(a))
a

39


C:\Users\Gerard\AppData\Local\Temp\ipykernel_15140\2924688107.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfpre['sd'] = [(row[1]['season'], row[1]['day']) for row in dfpre.iterrows()]
C:\Users\Gerard\AppData\Local\Temp\ipykernel_15140\2924688107.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  a = dfpre[dfpre['roll_type'] != 'GLITTER'].groupby(['element'])['roll', 'sd'].agg(["min", "max", "count"]).sort_values(by=[("roll", "min")])


roll                        sd                   \
                       min       max count       min        max count   
element                                                                 
Careful           0.006261  0.024896     5  (16, 29)   (18, 22)     5   
Ambitious         0.026552  0.038565     2  (17, 94)   (18, 49)     2   
Fire Eating       0.070789  0.072400     2  (16, 62)   (17, 94)     2   
Golden            0.087260  0.099422     3  (16, 62)   (18, 70)     3   
Squiddish         0.105174  0.126028     4  (16, 62)   (18, 49)     4   
Travelling        0.129190  0.149254     4  (16, 62)   (18, 22)     4   
Parasitic         0.154721  0.176920     6  (16, 93)   (18, 22)     6   
Clutch            0.188381  0.200338     6  (16, 62)   (18, 80)     6   
Chunky            0.216910  0.218152     2  (16, 29)   (17, 63)     2   
Smooth            0.231106  0.231106     1  (17, 71)   (17, 71)     1   
Repeating         0.257511  0.281972     8  (16, 29)   (18, 94)     8   
Fireproof         0.287601  0.304522     4  (16, 62)   (18, 80)     4   
Noise-Cancelling  0.309624  0.318620     3  (16, 29)   (17, 24)     3   
Gravity           0.339531  0.349605     4  (17, 24)   (17, 97)     4   
Cryogenic         0.360084  0.375468     3  (16, 62)   (18, 22)     3   
Plant-Based       0.390554  0.409678     3  (16, 93)   (18, 49)     3   
Metaphorical      0.411133  0.431818     4  (17, 24)   (18, 80)     4   
Actual            0.437211  0.451014     5  (17, 24)    (18, 8)     5   
Hearty            0.462894  0.484790     6  (16, 29)   (18, 49)     6   
Holey             0.489659  0.508237     5  (16, 29)   (18, 80)     5   
Limestone         0.517326  0.537014     2  (16, 29)   (17, 58)     2   
Mesh              0.540047  0.564028    10  (16, 29)   (18, 84)    10   
Rubber            0.564579  0.587750     6  (16, 93)   (18, 70)     6   
Aluminum          0.592024  0.612720     5  (17, 86)  (18, 114)     5   
Weird             0.621518  0.637004     5  (16, 62)   (18, 49)     5   
Smokey            0.654384  0.662073     2  (17, 58)   (17, 94)     2   
Charitable        0.669669  0.678011     4  (17, 58)   (18, 80)     4   
Slimy             0.696101  0.713274     6  (16, 93)   (18, 22)     6   
Greedy            0.721097  0.738297     5  (16, 29)   (18, 49)     5   
Passionate        0.751013  0.768623     4  (16, 29)   (18, 22)     4   
Hard              0.772025  0.793702     8  (16, 29)   (18, 64)     8   
Brambly           0.798884  0.801972     2  (16, 29)   (18, 80)     2   
Frosty            0.823944  0.841343     5  (16, 93)   (18, 80)     5   
Hot               0.848496  0.867207     4   (17, 6)   (18, 57)     4   
Cool              0.875812  0.890127     9  (17, 24)   (18, 80)     9   
Wooden            0.898149  0.918768     6  (16, 29)   (18, 80)     6   
Sharp             0.923684  0.943013     6  (16, 29)   (18, 80)     6   
Chaotic           0.949467  0.952452     2  (17, 24)   (17, 58)     2   
Lucky             0.974662  0.989363     4  (16, 29)   (18, 44)     4   

                      diff lower_bound upper_bound    lb    ub  
                                                                
element                                                         
Careful                NaN    0.000000    0.025641  True  True  
Ambitious         0.001656    0.025641    0.051282  True  True  
Fire Eating       0.032224    0.051282    0.076923  True  True  
Golden            0.014859    0.076923    0.102564  True  True  
Squiddish         0.005752    0.102564    0.128205  True  True  
Travelling        0.003162    0.128205    0.153846  True  True  
Parasitic         0.005466    0.153846    0.179487  True  True  
Clutch            0.011461    0.179487    0.205128  True  True  
Chunky            0.016572    0.205128    0.230769  True  True  
Smooth            0.012955    0.230769    0.256410  True  True  
Repeating         0.026404    0.256410    0.282051  True  True  
Fireproof         0.005629    0.282051    0.3

In [14]:
dfc2 = pd.concat([dfpre[dfpre['season'] != 15], pd.DataFrame(data=[
    {'season': 19, 'day': 100, 'roll_type': 'GLITTER', 'category': 'prefix', 'element': '???', 'roll': 0.99, 'prefix_index': 0},
    {'season': 19, 'day': 100, 'roll_type': 'GLITTER', 'category': 'prefix', 'element': '?????', 'roll': 0.55, 'prefix_index': 0},
    {'season': 19, 'day': 100, 'roll_type': 'GLITTER', 'category': 'prefix', 'element': '????', 'roll': 0.2, 'prefix_index': 0},
])])
a = dfc2[dfc2['roll_type'] == 'GLITTER'].groupby(['element'])['roll', 'sd'].agg(["min", "max", "count"]).sort_values(by=[("roll", "min")])
a['diff'] = a[('roll', 'min')] - a[('roll', 'max')].shift()
a['lower_bound'] = [n/len(a) for n in range(0,len(a))] 
a['upper_bound'] = [n/len(a) for n in range(1,len(a)+1)] 
a['lb'] = a['lower_bound'] <= a[('roll', 'min')]
a['ub'] = a['upper_bound'] >= a[('roll', 'max')]
print(len(a))
a

13


C:\Users\Gerard\AppData\Local\Temp\ipykernel_15140\905434645.py:6: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  a = dfc2[dfc2['roll_type'] == 'GLITTER'].groupby(['element'])['roll', 'sd'].agg(["min", "max", "count"]).sort_values(by=[("roll", "min")])


roll                        sd                      diff  \
                   min       max count       min       max count             
element                                                                      
Coasting      0.047566  0.048961     2  (17, 27)  (18, 85)     2       NaN   
Flickering    0.120699  0.120699     1  (18, 87)  (18, 87)     1  0.071739   
????          0.200000  0.200000     1       NaN       NaN     0  0.079301   
Parasitic     0.252490  0.252490     1  (16, 18)  (16, 18)     1  0.052490   
Repeating     0.371135  0.371135     1  (17, 63)  (17, 63)     1  0.118646   
Gravity       0.416979  0.416979     1  (18, 57)  (18, 57)     1  0.045844   
Metaphorical  0.482727  0.494332     2  (16, 22)  (18, 57)     2  0.065748   
?????         0.550000  0.550000     1       NaN       NaN     0  0.055668   
Inflatable    0.676970  0.676970     1  (16, 18)  (16, 18)     1  0.126970   
Charitable    0.741721  0.741721     1  (17, 82)  (17, 82)     1  0.064751   
Brambly       0.843394  0.843394     1  (16, 22)  (16, 22)     1  0.101672   
Cool          0.916626  0.916626     1  (16, 76)  (16, 76)     1  0.073232   
???           0.990000  0.990000     1       NaN       NaN     0  0.073374   

             lower_bound upper_bound    lb    ub  
                                                  
element                                           
Coasting        0.000000    0.076923  True  True  
Flickering      0.076923    0.153846  True  True  
????            0.153846    0.230769  True  True  
Parasitic       0.230769    0.307692  True  True  
Repeating       0.307692    0.384615  True  True  
Gravity         0.384615    0.461538  True  True  
Metaphorical    0.461538    0.538462  True  True  
?????           0.538462    0.615385  True  True  
Inflatable      0.615385    0.692308  True  True  
Charitable      0.692308    0.769231  True  True  
Brambly         0.769231    0.846154  True  True  
Cool            0.846154    0.923077  True  True  
???             0.923077    1.000000  True  True

In [17]:
denom = 37
dfc3 = pd.concat([dfpre[dfpre['season'] == 15], pd.DataFrame(data=[
    {'season': 15, 'day': 100, 'roll_type': 'GLITTER', 'category': 'prefix', 'element': '?' * n, 'roll': n/denom, 'prefix_index': 0} for n in range(2)
] + [
    {'season': 15, 'day': 100, 'roll_type': 'GLITTER', 'category': 'prefix', 'element': '?' * n, 'roll': n/denom, 'prefix_index': 0} for n in range(3, 4)
] + [
    {'season': 15, 'day': 100, 'roll_type': 'GLITTER', 'category': 'prefix', 'element': '?' * n, 'roll': n/denom, 'prefix_index': 0} for n in range(5, 15)
] + [
    {'season': 15, 'day': 100, 'roll_type': 'GLITTER', 'category': 'prefix', 'element': '?' * n, 'roll': n/denom, 'prefix_index': 0} for n in range(16, 21)
] + [
    {'season': 15, 'day': 100, 'roll_type': 'GLITTER', 'category': 'prefix', 'element': '?' * n, 'roll': n/denom, 'prefix_index': 0} for n in range(24, 30)
] + [
    {'season': 15, 'day': 100, 'roll_type': 'GLITTER', 'category': 'prefix', 'element': '?' * n, 'roll': n/denom, 'prefix_index': 0} for n in range(31, 32)
] + [
    {'season': 15, 'day': 100, 'roll_type': 'GLITTER', 'category': 'prefix', 'element': '?' * n, 'roll': n/denom, 'prefix_index': 0} for n in range(33, denom)
]
                                                        )])
a = dfc3[dfc3['roll_type'] == 'GLITTER'].groupby(['element'])['roll'].agg(["min", "max", "count"]).sort_values(by=[("min")])
a['diff'] = a[('min')] - a[('max')].shift(2)
a['lower_bound'] = [n/len(a) for n in range(0,denom)] 
a['upper_bound'] = [n/len(a) for n in range(1,denom+1)] 
a['lb'] = a['lower_bound'] <= a[( 'min')]
a['ub'] = a['upper_bound'] >= a[('max')]
print(len(a))
a

37


,min,max,count,diff,lower_bound,upper_bound,lb,ub
element,,,,,,,,
,0.000000,0.000000,1,NaN,0.000000,0.027027,True,True
?,0.027027,0.027027,1,NaN,0.027027,0.054054,True,True
Chaotic,0.078904,0.078904,1,0.078904,0.054054,0.081081,True,True
???,0.081081,0.081081,1,0.054054,0.081081,0.108108,True,True
?????,0.135135,0.135135,1,0.056231,0.108108,0.135135,True,True
Frosty,0.140581,0.140581,1,0.059500,0.135135,0.162162,True,True
??????,0.162162,0.162162,1,0.027027,0.162162,0.189189,True,True
???????,0.189189,0.189189,1,0.048608,0.189189,0.216216,True,True
????????,0.216216,0.216216,1,0.054054,0.216216,0.243243,True,True
